In [ ]:

%load_ext autoreload
%autoreload 2

In [13]:
#Get modules
from Incision import *
from Headcuts import *
from Erosion import *
from Invasives import *
#localpath = pathlib.Path.cwd()

In [ ]:
#INVASIVES#
#Get raw invasive data prior to 2024 that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_invasive_data_gdb()
#Get USFS rest service data spatially joined to SEZ's
usfsdf= get_USFSinvasive_data()
#Get new invasive data from sde collect
df = get_combined_survey_and_invasive_data()
#clean, process, grade data
process_grade_invasive(df, df, 2024)
#post data to sde.VECTOR.stage headcut table or csv
post_invasive(df, draft=True)

c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\Invasives.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usfsdf.rename(columns={'Assessment_Unit_Name': 'Assessment_Unit_Name', 'COMMON_NAME': 'plant_type'}, inplace=True)
c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\utils.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = np.nan
c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\utils.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

AttributeError: Can only use .dt accessor with datetimelike values

In [27]:
#HEADCUT#

#Get raw headcut data prior 2024 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allheadcut_data()
#Get new headcut data from sde collect
df = get_combined_survey_and_headcut_data()
#clean, process, grade data
process_grade_headcut(df, 2024)
#post data to sde.VECTOR.stage headcut table or csv
post_headcut(df, draft=True)


In [50]:
#INCISION#
#Get raw incision data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allincision_data()
#Get incision data from sde collect
df = get_combined_survey_and_incision_data()
#clean, process, grade data
process_grade_incision(df, 2024)
#post data to sde.VECTOR.stage incision table or csv
post_incision(df, draft=True)

In [ ]:
#EROSION#
#Get raw erosion data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
df= get_erosion_data()

#clean, process, grade data
process_grade_erosion(df, 2024)
#post data to sde.VECTOR.stage incision table or csv
post_erosion(df, draft=True)